In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from dataset import KittiVLOdomDataset
import tqdm

In [2]:
dataset = KittiVLOdomDataset("data/SemanticKITTI/dataset")

In [3]:
backbone = resnet18(pretrained=True)
backbone = nn.Sequential(*list(backbone.children())[:-2])  # remove final layers

In [4]:
dataloader = DataLoader(dataset, batch_size=4,
                        shuffle=False, num_workers=0)